In [43]:
%run ../../Utils/yp_utils.py

# Initial setup

In [44]:
paper_pmid = 23382077
paper_name = 'huang_paulovich_2013' 

In [45]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [46]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [47]:
original_data = pd.read_csv('raw_data/huang_paulovich_2013_hits.txt', header=None, names=['genes'])

In [48]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 14 x 1


In [49]:
original_data.head()

,genes
0,rad6
1,rad18
2,rad5
3,sgs1
4,top3


In [50]:
original_data['genes'] = original_data['genes'].astype(str)

In [51]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [52]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [53]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, orf]
Index: []


In [54]:
original_data['data'] = -1

In [55]:
original_data.set_index('orf', inplace=True)

In [56]:
original_data = original_data[['data']].copy()

In [57]:
original_data = original_data.groupby(original_data.index).mean()

In [58]:
original_data.shape

(14, 1)

# Load & process tested strains

In [59]:
tested = pd.read_excel('raw_data/Mat_a_obs_v4 0.xlsx', sheet_name='DATA')

In [60]:
tested.head()

,record no.,ORF name,Strain,Batch,Plate,Row,Col,Comment
0,338.0,YAL068C,BY4741,01_1,1.0,A,2.0,NaN
1,339.0,YAL067C,BY4741,01_1,1.0,A,3.0,NaN
2,340.0,YAL066W,BY4741,01_1,1.0,A,4.0,NaN
3,341.0,YAL065C,BY4741,01_1,1.0,A,5.0,NaN
4,345.0,YAL062W,BY4741,01_1,1.0,A,6.0,NaN


In [61]:
tested['orf'] = tested['ORF name'].astype(str)

In [62]:
tested['orf'] = clean_orf(tested['orf'])

In [63]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [64]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [65]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name Strain Batch  Plate  Row  Col Comment  orf
index_input                                                                
4705                NaN      NaN    NaN   NaN    NaN  NaN  NaN     NaN  NAN


In [66]:
tested = tested.loc[t,:]

In [67]:
tested_orfs = tested['orf'].unique()

In [68]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [69]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [70]:
data = original_data.copy()

In [71]:
dataset_ids = [145]
datasets = datasets.reindex(index=dataset_ids)

In [72]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [73]:
data.head()

dataset_id,145
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [74]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [75]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,145
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [76]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [77]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [78]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        145       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [79]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [83]:
# from IO.save_data_to_db3 import *

In [84]:
# save_data_to_db(data_all, paper_pmid)